# Анализ данных и алгоритмы машинного обучения

1. Математические библиотеки Python и их применение для анализа/предобработки данных - пример полиномиальной регрессии
1. Кластеризация данных и ее роль в отборе признаков и пре-процессинге
2. Анализ продуктовой корзины

__Для работы потребуются следующе датасеты:__
- [data/web_traffic.tsv](https://github.com/easyise/spec_python_courses/raw/master/python04-analysis/data/web_traffic.tsv)
- [data/store_data.csv](https://github.com/easyise/spec_python_courses/raw/master/python04-analysis/data/store_data.csv)

Для некоторых датасетов, которые мы будем сегодня загружать из интернета, может потребоваться порядка 500МБ дискового простанства.


__ВНИМАНИЕ__! Установите библиотеки scipy, sklearn и mlxtend: ```pip install scipy sklearn mlxtend```

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import scipy as sp
import statsmodels.api as sm

plt.rcParams['figure.figsize'] = (10.0, 10.0)
%matplotlib inline

### Пример применения полиномиальной регрессии для моделирования данных

Допустим, у нас есть ежечасная статистика веб-траффика по некоторому серверу. Нам нужно определить, когда по времени, с учетом текущей динамики, количество запросов превысит 50000/час, чтобы заранее проапгрейдить оборудование. Для этого мы попытаемся построить кривую с помощью полиномиальной регрессии и экстраполируя ее на будущее, определим крайний срок для апгрейда оборудования.

In [ ]:
web_traffic = pd.read_csv('data/web_traffic.tsv', sep='\t', header=None, names=['Hour', 'ReqsPerHour'], index_col='Hour')
web_traffic.head()

Разберемся с пропущенными данными:

In [ ]:
web_traffic.isnull().sum()

In [ ]:
web_traffic.dropna(inplace=True)

In [ ]:
web_traffic.shape

Напишем функцию для красивого отображения данных и моделей:

In [ ]:
def plot_models(x, y, models, mx=None, ymax=None, xmin=None):
    ''' plot input data '''
    
    colors = ['g', 'k', 'b', 'm', 'r']
    linestyles = ['-', '-.', '--', ':', '-']

    plt.figure(num=None, figsize=(10, 6))
    plt.clf()
    
    plt.scatter(x, y, s=10)
    
    plt.title("Web traffic over the last month")
    plt.xlabel("Time")
    plt.ylabel("Hits/hour")
    plt.xticks(
        [w * 7 * 24 for w in range(10)], ['week %i' % w for w in range(10)])

    if models:
        if mx is None:
            mx = np.linspace(0, x.shape[0], 1000)
        for model, style, color in zip(models, linestyles, colors):
            # print "Model:",model
            # print "Coeffs:",model.coeffs
            plt.plot(mx, model(mx), linestyle=style, linewidth=2, c=color)

        plt.legend(["d=%i" % m.order for m in models], loc="upper left")

    plt.autoscale(tight=True)
    plt.ylim(ymin=0)
    if ymax:
        plt.ylim(ymax=ymax)
    if xmin:
        plt.xlim(xmin=xmin)
    plt.grid(True, linestyle='-', color='0.75')
    
plot_models(web_traffic.index, web_traffic.ReqsPerHour, None)

In [ ]:
x, y = web_traffic.index, web_traffic.ReqsPerHour

fp1, res1, rank1, sv1, rcond1 = np.polyfit(x, y, 1, full=True)
print("Model parameters of fp1: %s" % fp1)
print("Error of the model of fp1:", res1)
f1 = sp.poly1d(fp1)

In [ ]:
plot_models(x, y, [f1])

Обучим еще несколько моделей с более высокой степенью многочлена:

In [ ]:
fp2, res2, rank2, sv2, rcond2 = np.polyfit(x, y, 2, full=True)
print("Model parameters of fp2: %s" % fp2)
print("Error of the model of fp2:", res2)
f2 = sp.poly1d(fp2)
f3 = sp.poly1d(np.polyfit(x, y, 3))
f10 = sp.poly1d(np.polyfit(x, y, 10))
f100 = sp.poly1d(np.polyfit(x, y, 100))

Нарисуем эти модели

In [ ]:
plot_models(x, y, [f1, f2, f3, f10, f100])

Замечаем, что точка перегиба нашего графика находится примерно на середине второй недели. Повторим обучение наших моделей с этим смещением.

In [ ]:
# fit and plot a model using the knowledge about inflection point
inflection = int(2.5 * 7 * 24)
xa = x[:inflection]
ya = y[:inflection]
xb = x[inflection:]
yb = y[inflection:]

fa = sp.poly1d(np.polyfit(xa, ya, 1))
fb = sp.poly1d(np.polyfit(xb, yb, 1))

In [ ]:
plot_models(x, y, [fa, fb])

In [ ]:
# fit and plot a model using the knowledge about inflection point
inflection = int(3.5 * 7 * 24)
xa = x[:inflection]
ya = y[:inflection]
xb = x[inflection:]
yb = y[inflection:]

fa = sp.poly1d(np.polyfit(xa, ya, 1))
fb = sp.poly1d(np.polyfit(xb, yb, 1))
plot_models(x, y, [fa, fb])

Нарисуем существующие модели с экстраполяцией в недалекое будущее (до 6-й недели с начала наблюдений)

In [ ]:
plot_models(
    x, y, [f1, f2, f3, f10, f100],
    mx=np.linspace(0 * 7 * 24, 6 * 7 * 24, 100),
    ymax=10000, xmin=0 * 7 * 24)

А также создадим несколько моделей, обученных на данных только после второй точки перегиба:

In [ ]:
fb1 = fb
fb2 = sp.poly1d(np.polyfit(xb, yb, 2))
fb3 = sp.poly1d(np.polyfit(xb, yb, 3))
fb10 = sp.poly1d(np.polyfit(xb, yb, 10))
fb100 = sp.poly1d(np.polyfit(xb, yb, 100))

Нарисуем их:

In [ ]:
plot_models(
    x, y, [fb1, fb2, fb3, fb10, fb100],
    mx=np.linspace(0 * 7 * 24, 6 * 7 * 24, 100),
    ymax=10000, xmin=0 * 7 * 24)

#### Оценим точность 

Напишем функцию которая считает среднеквадратичную ошибку для модели и посмотрим, на сколько точны наши первоначальные модели:

In [ ]:
def error(f, x, y):
    return np.sum((f(x) - y) ** 2)

print("Errors for the complete data set:")
for f in [f1, f2, f3, f10, f100]:
    print("Error d={}: {}" .format (f.order, error(f, x, y)))

...и оценим их же точность, но только после точки перегиба:

In [ ]:
print("Errors for only the time after inflection point")
for f in [f1, f2, f3, f10, f100]:
    print("Error d={}: {}" .format(f.order, error(f, xb, yb)))

...и теперь точность моделей, обученных после точки перегиба:

In [ ]:
print("Errors for only the time after inflection point")
for f in [fb1, fb2, fb3, fb10, fb100]:
    print("Error d=%i: %f" % (f.order, error(f, xb, yb)))

Выберем победительницей модель с полиномом в степени 2. Рассчитаем дату достижения предела в 50000 запросов.

In [ ]:
from scipy.optimize import fsolve
reached_max = fsolve(fb2 - 50000, x0=800) / (7 * 24)
print("50,000 hits/hour expected at week %f" % reached_max[0])

Нарисуем:

In [ ]:
plot_models(
    x, y, [fb2],
    mx=np.linspace(0 * 7 * 24, 8 * 7 * 24, 100),
    ymax=50000, xmin=0 * 7 * 24)

### Пример 2. Кластеризация 

В качестве визуальной оценки данных используется кластеризация.

In [ ]:
!pip install sklearn

In [ ]:
from sklearn.decomposition import PCA

Метод PCA (метод главных компонент) позволяет уменьшить размерность датасета до 2 (или 3).  Это позволяет визуально оценить "обучаемость" алгоритмов на этих данных. Также метод позволяет выяснить, из чего состоят итоговые компоненты. В основе этого метода лежит сингулярное разложение векторов (SVD). Рассмотрим на примере набора данных "Ирисы":

In [ ]:
from sklearn import datasets

iris = datasets.load_iris()
X, y = iris.data, iris.target

iris.data

In [ ]:
pca = PCA(n_components=2)
X_reduced = pca.fit_transform(X)

print("Meaning of the 2 components:")
for component in pca.components_:
    print(" + ".join("%.3f x %s" % (value, name)
                     for value, name in zip(component,
                                            iris.feature_names)))
plt.figure(figsize=(10,7))
plt.scatter(X_reduced[:, 0], X_reduced[:, 1], c=y, s=70, cmap='viridis')
plt.show()

In [ ]:
pca = PCA(n_components=3)
X_reduced = pca.fit_transform(X)

fig = plt.figure(figsize=(12,10))
ax = plt.axes(projection='3d')
ax.scatter( X_reduced[:, 0], X_reduced[:, 1], X_reduced[:, 2], c=y, cmap=plt.cm.get_cmap('viridis'))
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')

Рассмотрим на примере датасета "рукописные цифры". Здесь размерность уменьшена с 64 до 2.

In [ ]:
from sklearn.datasets import load_digits

digits = load_digits()
X = digits.data
y = digits.target


pca = PCA(n_components=2)
X_reduced = pca.fit_transform(X)

print('Projecting %d-dimensional data to 2D' % X.shape[1])

plt.figure(figsize=(12,10))
plt.scatter(X_reduced[:, 0], X_reduced[:, 1], c=y, 
            edgecolor='none', alpha=0.7, s=40,
            cmap=plt.cm.get_cmap('nipy_spectral', 10))
plt.colorbar()
plt.show()

In [ ]:
pca = PCA(n_components=3)
X_reduced = pca.fit_transform(X)

fig = plt.figure(figsize=(12,10))
ax = plt.axes(projection='3d')
ax.scatter( X_reduced[:, 0], X_reduced[:, 1], X_reduced[:, 2], c=y, cmap=plt.cm.get_cmap('nipy_spectral'))
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
plt.legend()
plt.show()

Пример кластеризации по t-SNE.

In [ ]:
from sklearn.manifold import TSNE

plt.figure(figsize=(20,20))

tsne = TSNE()
X_embedded = tsne.fit_transform(X)

sns.scatterplot(x=X_embedded[:,0], y=X_embedded[:,1], hue=y, legend='full', palette=sns.color_palette("bright", 10))

In [ ]:
tsne = TSNE(n_components=3)
X_embedded = tsne.fit_transform(X)

ig = plt.figure(figsize=(12,10))
ax = plt.axes(projection='3d')
ax.scatter( X_embedded[:, 0], X_embedded[:, 1], X_embedded[:, 2], c=y, cmap=plt.cm.get_cmap('nipy_spectral'))
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')

### Пример 3. Анализ покупательской корзины

Это пример "майнинга данных" - поиска зависимостей и закономерностей в массиве данных. "Анализ покупательской корзины" - довольно часто использующийся метод при построении рекомендательных систем. Базовый алгоритм называется ```Apriori```, он был предложен в 1994 году.

Назначение алгоритма - поиск часто встречающихся подмножеств. Он оперирует следующими понятиями:
 - "суппорт" $Support A$ - вероятность покупки товара A, ее можно вычислить как отношение количества покупок A к общему количеству покупок.
 - "конфидент" $Conf  A{\rightarrow}B$  - вычисляется для пары товаров A и B как отношение случаев совместного приобретения этих товаров к покупкам артикула A
 - "подъем" $Lift  A{\rightarrow}B$  - это отношение вероятности приобретения пары товаров A и B к вероятности приобретения товара A. Или попросту говоря, отношение "конфидента" к "суппорту".
 
Давайте "вытащим" из датасета с данными о покупках в некотором французском супермаркете самые популярные товары и их сочетания с помощью алгоритма ```Apriori```.


Прочитаем датасет и посмотрим на его содержимое:

In [ ]:
df_store = pd.read_csv('data/store_data.csv', header=None)
print(df_store.shape)
df_store.head()

Посмотрим на список артикулов и их количество:

In [ ]:
df_store.stack().value_counts()

На базе исходного dataframe создадим dataframe, в котором признаки приобретения того или иного товара станут булевыми dummy признаками. Для этого мы ему сначала сделаем ```stack()```, получим dummy-признаки, а затем сгруппируем по индексу первого уровня со взятием максимума:

In [ ]:
df_dummies = pd.get_dummies(df_store.stack()).groupby(level=0).max()
df_dummies.head()

Запустим алгоритм Apriori и получим список наиболее часто приобретаемых товаров и их сочетаний:

In [ ]:
from mlxtend.frequent_patterns import apriori

df_apriori = apriori(df_dummies, min_support=0.01, use_colnames=True)
df_apriori

Отсортируем список:

In [ ]:
df_apriori.sort_values('support', ascending=False)

Теперь получим список самых популярных сочетаний для 2-х и 3-х товаров в корзине:

In [ ]:
df_apriori['item_count'] = df_apriori['itemsets'].apply(len)
df_apriori[ df_apriori.item_count >=3 ].sort_values('support', ascending=False)[:20]

4. Нейронные сети

В рамках данной демонстрации мы, конечно, не будем углубляться во все тонкости обучения, настройки и использования нейросетей, это тема для отдельного курса. Ниже - просто пример использования сети imagenet в задаче распознавания образов: 

In [ ]:
!pip install tensorflow keras

In [ ]:
import keras
keras.__version__

In [ ]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input, decode_predictions

model = VGG16(weights='imagenet')

In [ ]:
# Загрузим изображение
img_path = 'data/creative_commons_elephant.jpg'

img_full = image.load_img(img_path)
plt.imshow(img_full)

In [ ]:
# выполним необходимый пре-процессинг изображения и вызовем функцию predict()

# `img` is a PIL image of size 224x224
img = image.load_img(img_path, target_size=(224, 224))

# `x` is a float32 Numpy array of shape (224, 224, 3)
x = image.img_to_array(img)

# We add a dimension to transform our array into a "batch"
# of size (1, 224, 224, 3)
x = np.expand_dims(x, axis=0)

# Finally we preprocess the batch
# (this does channel-wise color normalization)
x = preprocess_input(x)

preds = model.predict(x)
print('Predicted:', decode_predictions(preds, top=3)[0])

In [ ]:
# узнать об архитектуре и свойствах модели можно с помощью функции summary()

model.summary()